# Основы Pandas

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\UserOfPC\Documents\Бруноям окт_2023\ds_salaries_mod.csv')

In [3]:
df.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847.0,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000.0,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500.0,US,100,US,S


In [4]:
df.work_year.value_counts()  # Значения по количеству

2023    1785
2022    1664
2021     230
2020      76
Name: work_year, dtype: int64

### Основная статистика

In [5]:
df.salary_in_usd.describe()

count      2871.000000
mean     137764.306513
std       63324.240058
min        5132.000000
25%       95000.000000
50%      135000.000000
75%      175000.000000
max      450000.000000
Name: salary_in_usd, dtype: float64

In [6]:
df[df.work_year == 2023]['salary_in_usd'].mean()

149508.4592809978

In [7]:
df[df.work_year == 2022]['salary_in_usd'].mean()

133438.8639562158

In [8]:
df['same_country'] = (df['company_location'] == df['employee_residence'])

In [9]:
df['same_country'].value_counts()

True     3659
False      96
Name: same_country, dtype: int64

### Группировка

In [10]:
df.groupby('work_year')['salary_in_usd'].mean()

work_year
2020     89881.483333
2021     92781.644970
2022    133438.863956
2023    149508.459281
Name: salary_in_usd, dtype: float64

In [11]:
df.groupby('experience_level')['salary_in_usd'].agg(['count', 'mean', 'median'])

,count,mean,median
experience_level,,,
EN,239,79363.953975,71907.0
EX,76,198120.828947,200000.0
MI,603,104090.585406,100000.0
SE,1953,152959.310292,146200.0


## Домашнее задание

### Easy

Изучите распределение размеров компаний (company_size) с помощью метода .value_counts(). Опишите своими словами, что вы увидели.

In [12]:
df_company_size = df.company_size.value_counts()
df_company_size = df_company_size.to_frame()
df_company_size

,company_size
M,3153
L,454
S,148


In [13]:
df_company_size['company_size, %'] = round((df_company_size['company_size'] / df_company_size['company_size'].sum())*100, 1)
df_company_size

,company_size,"company_size, %"
M,3153,84.0
L,454,12.1
S,148,3.9


Вывод: в рассматриваемом датасете преобладают компании среднего размера, их 84%. Больших компаний - 12% и малых компаний - всего 4%.

### Normal

Сделайте выборку за 2023 год. Постройте по ней сводную таблицу, сгруппированную по размеру компании-работодателя (company_size), со средней и медианной зарплатой в долларах. Не забывайте также указать количество.

In [14]:
df_salary_size_2023 = df[df['work_year'] == 2023].groupby(df['company_size'])['salary_in_usd']  \
                           .agg(['count', 'mean','median'])  \
                           .reset_index()  \
                           .sort_values('median', ascending=False)
df_salary_size_2023['median-mean, %'] =  round(((df_salary_size_2023['median'] - df_salary_size_2023['mean']) / df_salary_size_2023['mean'])*100, 2)
df_salary_size_2023

,company_size,count,mean,median,"median-mean, %"
1,M,1275,151126.316078,145000.0,-4.05
0,L,71,138622.845070,126100.0,-9.03
2,S,17,73632.647059,53654.0,-27.13


Из сводной таблицы можно заметить, что в малых компаниях и средняя и медианная зарплаты сильно меньше, чем в крупных и средних компаниях, а также средняя з/п выше медианной на 27%, это говорит нам о достаточно сильном разбросе зарплат (возможно есть выбросы). Самые большие зарплаты видим в средних компаниях, причем медианная и средняя з/п примерно одинаковые. В группе крупных компаний также медианная з/п меньше средней, но уже на 9%.

Аналогично исследуйте 2022 год. Есть ли отличия?

In [15]:
df_salary_size_2022 = df[df['work_year'] == 2022].groupby(df['company_size'])['salary_in_usd']  \
                           .agg(['count', 'mean','median'])  \
                           .reset_index()  \
                           .sort_values('median', ascending=False)
df_salary_size_2022['median-mean, %'] =  round(((df_salary_size_2022['median'] - df_salary_size_2022['mean']) / df_salary_size_2022['mean'])*100, 2)
df_salary_size_2022

,company_size,count,mean,median,"median-mean, %"
1,M,1096,137245.859489,135000.0,-1.64
0,L,143,120581.482517,120000.0,-0.48
2,S,40,75092.325000,58000.0,-22.76


Можно заметить, что в средних и крупных компаниях наблюдается рост средних зарплат в 2023г. по отношению к 2022г., в малых компаниях, наоборот, снижение. Общая тенденция в размерах зарплат по размерам компаний сохранилась. Однако, можно заметить, что количество крупных и малых компаний было почти в два раза больше, чем в 2023 г. Также видим, что в средних и крупных компаниях медианная и средняя зарплаты меньше отличались между собой. 

## Hard

Для начала попробуйте восстановить пропущенные значения salary_in_usd.

Один из способов:

- создайте дополнительный столбец с "курсом доллара" для каждого наблюдения;

- проверьте, менялся ли курс каждой валюты в течение года;

- по возможности примените этот курс для расчета пропущенного значения;

- если остались пропуски, которые нельзя заполнить, удалите их.

Оставляйте комментарии в коде своего решения.

In [16]:
# Добавим столбец курса валюты
df['exchange_rate'] = round( df['salary_in_usd'] / df['salary'], 4)
df.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,same_country,exchange_rate
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847.0,ES,100,ES,L,True,1.0731
1,2023,MI,CT,ML Engineer,30000,USD,30000.0,US,100,US,S,True,1.0000
2,2023,MI,CT,ML Engineer,25500,USD,25500.0,US,100,US,S,True,1.0000


In [17]:
# Посмотрим сколько всего пропущенных значений в exchange_rate
df['exchange_rate'].isna().sum()

884

In [18]:
# Посмотрим на сводую таблицу по всем курсам валют в каждом году
df.groupby(['work_year', 'salary_currency'])['exchange_rate'].agg(['max', 'min', 'count'])

max     min  count
work_year salary_currency                       
2020      CAD                 NaN     NaN      0
          DKK              0.1530  0.1530      1
          EUR              1.1405  1.1404     21
          GBP              1.2826  1.2826      2
          HUF                 NaN     NaN      0
          INR              0.0135  0.0135      4
          JPY              0.0094  0.0094      1
          MXN                 NaN     NaN      0
          USD              1.0000  1.0000     31
2021      AUD              0.7505  0.7505      2
          BRL              0.1854  0.1854      1
          CAD              0.7976  0.7976      4
          CLP              0.0013  0.0013      1
          DKK              0.1589  0.1589      1
          EUR              1.1821  1.1820     34
          GBP              1.3755  1.3755     11
          HUF              0.0033  0.0033      1
          INR              0.0135  0.0135     15
          JPY              0.0091  0.0091      2
          PLN              0.2589  0.2589      2
          SGD              0.7441  0.7441      3
          TRY              0.1121  0.1121      2
          USD              1.0000  1.0000     90
2022      AUD              0.6931  0.6931      4
          BRL              0.1935  0.1935      1
          CAD              0.7681  0.7681      8
          CHF              1.0470  1.0470      3
          CZK              0.0428  0.0428      1
          EUR              1.0507  1.0506     91
          GBP              1.2313  1.2313     66
          HUF                 NaN     NaN      0
          INR              0.0127  0.0127     15
          PLN              0.2241  0.2241      2
          SGD              0.7251  0.7251      1
          USD              1.0000  1.0000   1087
2023      AUD              0.6832  0.6832      1
          BRL              0.1925  0.1925      1
          CAD              0.7393  0.7393      3
          CHF              1.0812  1.0812      1
          EUR              1.0731  1.0730     35
          GBP              1.2152  1.2152     43
          HKD                 NaN     NaN      0
          ILS              0.2826  0.2826      1
          INR              0.0122  0.0122      9
          PLN              0.2279  0.2279      1
          SGD              0.7502  0.7502      1
          THB              0.0295  0.0295      1
          USD              1.0000  1.0000   1266

Курс каждой валюты постоянен в течении каждого года, поэтому в пропущенных ячейках укажем значение курса данной валюты в указанном году. Однако видим 5 строк, в которых отсутствуют значения и которые являются уникальными в году (count = 0).

In [20]:
""" Сгруппируем данные по году и валюте и заменим отсутствующие данные о курсе максимальным курсом валюты в нужном году 
 (т.к. курс постоянный в году, то не важно максимумом или минимумом заменять). 
 fillna +  groupby + transform """

df['exchange_rate'].fillna(df.groupby(['work_year', 'salary_currency'])['exchange_rate'].transform('max'), inplace=True)

In [23]:
# Посмотрим на результат на примере выборки по валюте EUR
df.query('salary_currency == "EUR"').head(5)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,same_country,exchange_rate
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847.0,ES,100,ES,L,True,1.0731
82,2023,MI,FT,Applied Machine Learning Engineer,65000,EUR,69751.0,IN,100,DE,S,False,1.0731
84,2023,MI,FT,Machine Learning Engineer,90000,EUR,NaN,NL,100,NL,L,True,1.0731
91,2023,SE,FT,Data Scientist,45000,EUR,48289.0,ES,0,ES,M,True,1.0731
92,2023,SE,FT,Data Scientist,36000,EUR,38631.0,ES,0,ES,M,True,1.0731


Теперь, когда у нас есть недостающие курсы валют, заменим пустые значения в колонке salary_in_usd по формуле (salary / exchange_rate)

In [25]:
# Используем нампай метод where, который NaN заменит произведением salary * exchange_rate
df['salary_in_usd'] = np.where(pd.isna(df['salary_in_usd']), (df['salary'] * df['exchange_rate']), df['salary_in_usd'])
df['salary_in_usd'] = df['salary_in_usd'].round(1)

Посмотрим на результат замены по той же выборке валюта = EUR

In [28]:
df.query('salary_currency == "EUR"').head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,same_country,exchange_rate
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847.0,ES,100,ES,L,True,1.0731
82,2023,MI,FT,Applied Machine Learning Engineer,65000,EUR,69751.0,IN,100,DE,S,False,1.0731
84,2023,MI,FT,Machine Learning Engineer,90000,EUR,96579.0,NL,100,NL,L,True,1.0731
91,2023,SE,FT,Data Scientist,45000,EUR,48289.0,ES,0,ES,M,True,1.0731
92,2023,SE,FT,Data Scientist,36000,EUR,38631.0,ES,0,ES,M,True,1.0731


Проверим сколько пустых значений exchange_rate осталось после замены.

In [29]:
df['exchange_rate'].isna().sum()

5

In [30]:
df[df['salary_in_usd'].isna() == True]

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,same_country,exchange_rate
80,2023,MI,FT,Data Scientist,510000,HKD,NaN,HK,0,HK,L,True,NaN
2358,2022,EN,FT,Data Scientist,6600000,HUF,NaN,HU,100,HU,M,True,NaN
3419,2020,SE,FT,Machine Learning Manager,157000,CAD,NaN,CA,50,CA,L,True,NaN
3594,2020,SE,FT,Data Engineer,720000,MXN,NaN,MX,0,MX,S,True,NaN
3646,2020,MI,FT,Data Scientist,11000000,HUF,NaN,HU,50,HU,L,True,NaN


Данные строки можно удалить, потому что корректно восстановить не предоставляется возможным (не известен курс валюты в данном году).

In [31]:
df = df.dropna(subset='salary_in_usd')
df['exchange_rate'].isna().sum()

0

Дополнительно: попробуйте свести данные в столбце job_title к 4 основным профессиям (Data Scientist, Data Analyst, Data Engineer, ML Engineer), где это возможно.

In [33]:
df.groupby(df['job_title'])['salary'].count().sort_values(ascending=False).head(30)

job_title
Data Engineer                               1039
Data Scientist                               837
Data Analyst                                 612
Machine Learning Engineer                    289
Analytics Engineer                           103
Data Architect                               101
Research Scientist                            82
Data Science Manager                          58
Applied Scientist                             58
Research Engineer                             37
ML Engineer                                   34
Data Manager                                  29
Machine Learning Scientist                    26
Data Science Consultant                       24
Data Analytics Manager                        22
Computer Vision Engineer                      18
AI Scientist                                  16
BI Data Analyst                               15
Business Data Analyst                         15
Data Specialist                               14
BI Develop

Для того, чтобы переименовать специальности будем использовать схему: 

- **df.job_title.str.contains(word[ ])** - делаем "маску", отмечая как True те строки, у которых в столбце job_title содержит слово из переменной word (список, обращаемся к каждому слову по индексу).

- **df.loc["маска",'job_title'] = "Новая специальность"** - берём те строки, у которых в "маске" стоит True и в них берём столбец job_title, пишем в этот столбец нашу строку во все эти выбранные строки

In [34]:
word = ['Analyst', 'Analytics', 'BI', 'Business Intelligence']

df.loc[(df.job_title.str.contains(word[0])) \
       | (df.job_title.str.contains(word[1]))  \
       | (df.job_title.str.contains(word[2]))  \
       | (df.job_title.str.contains(word[3])), \
       'job_title'] = 'Data Analyst'

In [35]:
word = ['Science', 'Scientist']

df.loc[(df.job_title.str.contains(word[0])) \
       | (df.job_title.str.contains(word[1])), \
       'job_title'] = 'Data Scientist'

In [36]:
word = ['Machine Learning', 'ML Engineer', 'Computer Vision', 'Research Engineer', 'AI', 
        'Deep Learning', 'ETL', 'NLP', 'MLOps', 'Autonomous Vehicle Technician']

df.loc[(df.job_title.str.contains(word[0])) \
       | (df.job_title.str.contains(word[1]))  \
       | (df.job_title.str.contains(word[2]))  \
       | (df.job_title.str.contains(word[3]))  \
       | (df.job_title.str.contains(word[4]))  \
       | (df.job_title.str.contains(word[5]))  \
       | (df.job_title.str.contains(word[6]))  \
       | (df.job_title.str.contains(word[7]))  \
       | (df.job_title.str.contains(word[8]))  \
       | (df.job_title.str.contains(word[9])), \
       'job_title'] = 'ML Engineer'

In [37]:
word = ['Data Engineer', 'Data Architect', 'Data Management', 'DevOps Engineer', 'Data Strategist', 
        'Data Modeler', 'Data Lead', 'Database Engineer', 'Infrastructure Engineer', 'Head of Data', 
        'Data Operations Engineer', 'Data Specialist', 'Data Manager']

df.loc[(df.job_title.str.contains(word[0]))
       | (df.job_title.str.contains(word[1]))  \
       | (df.job_title.str.contains(word[2]))  \
       | (df.job_title.str.contains(word[3]))  \
       | (df.job_title.str.contains(word[4]))  \
       | (df.job_title.str.contains(word[5]))  \
       | (df.job_title.str.contains(word[6]))  \
       | (df.job_title.str.contains(word[7]))  \
       | (df.job_title.str.contains(word[8]))  \
       | (df.job_title.str.contains(word[9]))  \
       | (df.job_title.str.contains(word[10]))  \
       | (df.job_title.str.contains(word[11]))  \
       | (df.job_title.str.contains(word[12])), \
       'job_title'] = 'Data Engineer'

In [38]:
df['job_title'].value_counts()

Data Engineer     1253
Data Scientist    1178
Data Analyst       840
ML Engineer        479
Name: job_title, dtype: int64

Сделайте выборку за 2023 год. Постройте по ней сводную таблицу, сгруппированную по профессии (job_title), со средней и медианной зарплатой в долларах. Не забывайте также указать количество. (Если у вас не получилось привести в порядок столбец job_title, выведите только 4 самых распространенных профессии.)

In [40]:
df[df['work_year'] == 2023].groupby(df['job_title'])['salary_in_usd']  \
                           .agg(['count', 'mean','median'])  \
                           .sort_values('median', ascending=False)  \
                           .reset_index()

,job_title,count,mean,median
0,Data Scientist,525,163251.657143,157750.0
1,ML Engineer,252,162696.968254,150225.0
2,Data Engineer,602,150592.721096,141600.0
3,Data Analyst,405,120043.847901,113000.0


Теперь давайте получше категоризируем зарплаты. Изучите самостоятельно документацию к функциям pandas cut и qcut. Используйте их, чтобы разбить зарплаты на три или больше категорий (например: Low, Medium, High) и запишите их в столбец salary_range.

Сделайте сводную таблицу по профессиям за 2023 год, где будут выведены:

число работников в профессии;
самая часто встречающаяся категория зарплаты.
Подсказка (нажмите, чтобы посмотреть): lambda x : pd.Series.mode(x)

In [41]:
df['salary_range'] = pd.cut(df['salary_in_usd'], 3, labels=["Low", "Medium", "High"])

In [42]:
df[df['work_year'] == 2023].groupby(df['job_title'])['salary_range']  \
                           .agg(['count', pd.Series.mode])  \
                           .reset_index()

,job_title,count,mode
0,Data Analyst,405,Low
1,Data Engineer,602,Low
2,Data Scientist,525,Medium
3,ML Engineer,252,Low


Проверьте себя: совпадают ли категории с реальными цифрами?

In [43]:
df_salary_range = df[df['work_year'] == 2023].groupby(['job_title', 'salary_range'])['salary_in_usd']  \
                           .agg(['count', 'mean'])
df_salary_range

count           mean
job_title      salary_range                      
Data Analyst   Low             309   99153.477023
               Medium           95  185203.515789
               High              1  385000.000000
Data Engineer  Low             345  111515.055362
               Medium          250  199714.176000
               High              7  322225.714286
Data Scientist Low             249  110683.880723
               Medium          266  205600.863534
               High             10  345700.400000
ML Engineer    Low             132  114739.393939
               Medium          114  209693.210526
               High              6  324835.000000

Данные по категориям совпадают: самая часто встречающаяся категория зарплаты "Low" у специалистов Data Analyst, Data Engineer, ML Engineer. У специалистов Data Scientist наиболее многочисленная группа з/п - "Medium", обгоняет группу "Low" на 7%.